<a href="https://colab.research.google.com/github/esnue/ThesisAllocationSystem/blob/main/contextualized_topic_modeling" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Contextualized Topic Models with Wikipedia Documents

(last updated 11-01-2021)

In this tutorial, we are going to use contextualized topic modeling to get topics out of a collections of articles you will upload here.

## Topic Models 

Topic models allow you to discover latent topics in your documents in a completely unsupervised way. Just use your documents and get topics out.

## Contextualized Topic Models

![](https://raw.githubusercontent.com/MilaNLProc/contextualized-topic-models/master/img/logo.png)

What are Contextualized Topic Models? **CTMs** are a family of topic models that combine the expressive power of BERT embeddings with the unsupervised capabilities of topic models to get topics out of documents.

## Python Package

You can find our package [here](https://github.com/MilaNLProc/contextualized-topic-models).

![https://travis-ci.com/MilaNLProc/contextualized-topic-models](https://travis-ci.com/MilaNLProc/contextualized-topic-models.svg) ![https://pypi.python.org/pypi/contextualized_topic_models](https://img.shields.io/pypi/v/contextualized_topic_models.svg) ![https://pepy.tech/badge/contextualized-topic-models](https://pepy.tech/badge/contextualized-topic-models)




# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Data

We are going to need some data. You should upload a file with one document per line. We assume you haven't run any preprocessing script.

However, if you want to first test the model without uploading your data, you can simply use the test file I'm putting here

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

In [ ]:
!head -n 1 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry


In [ ]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt" # EDIT THIS WITH THE FILE YOU UPLOAD

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

file = pd.read_csv("/content/drive/MyDrive/ThesisAllocationSystem/data_final/train-papers-final.csv")
#transform content column row values into a list
text_file = file['Content'][1:].values.tolist()

print(text_file[5])
print(type(text_file))

b"Public Administration Reform in Europe  \n\nViews and Experiences from Senior Executives in 10 Countries \n\n \nWP3 Cross-national Report as part of the COCOPS Research Project \n\n \n\n  \n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nGerhard Hammerschmid \nSteven Van de Walle \n\nRhys Andrews \nPhilippe Bezes \nAnja G\xf6rnitz \nAnca Oprisor \n\nVid timac \n\n \n\nJuly 2013 \n\n \n\n \n\n \n\n \n\n \n\nCoordination for Cohesion in the Public Sector of the Future (COCOPS): www.cocops.eu \n\n  \n\n \n\nhttp://www.cocops.eu/\n\n\nCOCOPS Work Package 3  Cross-national Report, July 2013 Page 2 \n\nAbout COCOPS \n\nThe COCOPS project (Coordinating for Cohesion in the Public Sector of the Future) seeks to \n\ncomparatively and quantitatively assess the impact of New Public Management-style reforms in \n\nEuropean countries, drawing on a team of European public administration scholars from 11 \n\nuniversities in 10 countries. It analyses the impact of reforms i

# Installing Contextualized Topic Models

Now, we install the contextualized topic model library

In [3]:
%%capture
!pip install contextualized-topic-models==1.8.1
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Restart the Notebook

For the changes to take effect, we now need to restart the notebook.

From the Menu:

+ Runtime → Restart Runtime

## Importing what we need

In [5]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_list, TopicModelDataPreparation
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
from gensim.models import ldamodel 
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk
import os
import pickle

## Preprocessing

Let's pass our file with preprocess data to our text handler object. This object takes care of creating the bag of words for you.

Why do we use the **preprocessed text** here? we need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [8]:
nltk.download('stopwords')

documents = text_file
#documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, "english")
preprocessed_documents_for_bow, unpreprocessed_corpus_for_contextual, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


You might want to pickle the **training_dataset** object to avoid recomputing the BoW multiple times

In [9]:
qt = TopicModelDataPreparation("bert-base-nli-mean-tokens")

training_dataset = qt.create_training_set(unpreprocessed_corpus_for_contextual, preprocessed_documents_for_bow)

100%|██████████| 405M/405M [00:14<00:00, 27.0MB/s]


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

## Training our Contextualized Topic Model

Finally, we can fit our new topic model. We will ask the model to find 50 topics in our collection (n_component parameter of the CTM object).

In [10]:
ctm = CombinedTM(input_size=len(qt.vocab), bert_input_size=768, n_components=25, num_epochs=15)

ctm.fit(training_dataset) 

Epoch: [15/15]	 Seen Samples: [12150/12150]	Train Loss: 13131.625771604939	Time: 0:00:00.376945: : 15it [00:05,  2.61it/s]


# Topics

After training, now it is the time to look at our topics: we can use the 

```
get_topic_lists
```

function to get the topics. It also accepts a parameter that allows you to select how many words you want to see for each topic.

If you look at the topics, you will see that they all make sense and are representative of a collection of documents that comes from Wikipedia (general knowledge).

In [11]:
ctm.get_topic_lists(5)

[['party', 'parties', 'political', 'election', 'elections'],
 ['member', 'policy', 'energy', 'financial', 'european'],
 ['firms', 'firm', 'nn', 'productivity', 'nd'],
 ['public', 'political', 'policy', 'actors', 'making'],
 ['education', 'vocational', 'training', 'dual', 'vet'],
 ['und', 'zu', 'der', 'den', 'die'],
 ['eu', 'states', 'european', 'asylum', 'protection'],
 ['law', 'rights', 'court', 'legal', 'human'],
 ['rights', 'human', 'law', 'international', 'courts'],
 ['online', 'people', 'media', 'social', 'new'],
 ['social', 'people', 'organization', 'society', 'organizations'],
 ['german', 'age', 'fertility', 'germany', 'first'],
 ['data', 'results', 'time', 'used', 'increase'],
 ['market', 'costs', 'prices', 'economic', 'model'],
 ['market', 'price', 'costs', 'energy', 'cost'],
 ['women', 'results', 'sample', 'however', 'employment'],
 ['firms', 'firm', 'productivity', 'et', 'paper'],
 ['states', 'legal', 'general', 'eu', 'tandfonline'],
 ['asked', 'steps', 'regular', 'nresults'

### Let's find our documents' topics

Ok now we can take a document and see which topic has been asigned to it. Results will obviously change with respect to the documents you are using. For example, let's predict the topic of the first preprocessed document that is taling about a peninsula.

In [12]:
topics_predictions = ctm.get_doc_topic_distribution(training_dataset, n_samples=20) # get all the topic predictions

Sampling: [20/20]: : 20it [00:06,  2.88it/s]


In [15]:
preprocessed_documents_for_bow[0] # see the text of our preprocessed document

'nsee discussions author publication https www researchgate net publication incentive preferences issues strategies local government first evidence narticle authors including nsome authors publication also working related projects open innovative governments view project public sector future view project publications nsee profile school governance publications nsee profile nall content following page september nthe user downloaded nhttps www researchgate net publication motivation incentive preferences issues strategies local government first evidence austria enrichid rgreq xxx enrichsource el esc publicationcoverpdf nhttps www researchgate net publication motivation incentive preferences issues strategies local government first evidence austria enrichid rgreq xxx enrichsource el esc publicationcoverpdf nhttps www researchgate net project open innovative governments enrichid rgreq xxx enrichsource el esc publicationcoverpdf nhttps www researchgate net project public sector future enric

In [17]:
import numpy as np
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document
topic_number

24

In [20]:
ctm.get_topic_lists(5)[topic_number] #and the topic should and could be about public management related things

['civil', 'groups', 'war', 'peace', 'society']